## Application of BPR on Zazzle Data

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import sys
import itertools
import numpy as np
import pandas as pd
from subprocess import call
from pybpr import *
import scipy.sparse as sp
from functools import reduce
import matplotlib.pyplot as plt
from functools import partial

SyntaxError: invalid syntax (utils.py, line 257)

In [4]:
import sys
sys.path.append('/home/rsandhu/zazzle/rsandhu/pybpr')

## Put Zazzle data in UserItemInteration Class object

In [3]:
DATA_DIR = '/kfs2/projects/zazzle/rsandhu/pybpr/examples/data/zazzle_big'
df_v = pd.read_parquet(os.path.join(DATA_DIR, 'view_data.parquet'))
df_c = pd.read_parquet(os.path.join(DATA_DIR, 'click_data.parquet'))
df_o = pd.read_parquet(os.path.join(DATA_DIR, 'order_data.parquet'))
df_v_not_c = pd.read_parquet(os.path.join(
    DATA_DIR, 'viewed_not_clicked_data.parquet'))
df_c_not_o = pd.read_parquet(os.path.join(
    DATA_DIR, 'clicked_not_ordered_data.parquet'))

In [4]:
df_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226069034 entries, 0 to 226069033
Data columns (total 5 columns):
 #   Column           Dtype   
---  ------           -----   
 0   user_member_id   category
 1   date_created     int64   
 2   product_id       int64   
 3   user_member_idx  int64   
 4   product_idx      int64   
dtypes: category(1), int64(4)
memory usage: 7.6 GB


In [5]:
df_v.user_member_id.nunique(), df_v.product_id.nunique()
df_v.user_member_idx.nunique(), df_v.product_idx.nunique()

(593578, 4340406)

In [6]:
# viewed not clicked
num_users = df_v.user_member_id.nunique()
num_items = df_v.product_id.nunique()

test_ratio = 0.0
data_viewed_not_clicked = UserItemInteractions(
    users_index=df_v_not_c['user_member_idx'],
    items_index=df_v_not_c['product_idx'],
    num_users=num_users,
    num_items=num_items
)
data_viewed_not_clicked.generate_train_test(user_test_ratio=test_ratio)

# clicked
data_clicked = UserItemInteractions(
    users_index=df_c['user_member_idx'],
    items_index=df_c['product_idx'],
    num_users=num_users,
    num_items=num_items
)
data_clicked.generate_train_test(user_test_ratio=test_ratio)

# clicked not ordered
data_clicked_not_ordered = UserItemInteractions(
    users_index=df_c_not_o['user_member_idx'],
    items_index=df_c_not_o['product_idx'],
    num_users=num_users,
    num_items=num_items
)
data_clicked_not_ordered.generate_train_test(user_test_ratio=test_ratio)

# ordered
data_ordered = UserItemInteractions(
    users_index=df_o['user_member_idx'],
    items_index=df_o['product_idx'],
    num_users=num_users,
    num_items=num_items
)
data_ordered.generate_train_test(user_test_ratio=test_ratio)


----UserItemInteractions--Sample
# of users (active/total): 591695/593578
# of items (active/total): 4340403/4340406
# of interactions: 221401173
Sparsity in the UI mat: 8.6e-05
Memory used by sparse UI mat: 211.14 MB
Generating train-test split..Warning: Test matrix is set as empty/all-zeros

----UserItemInteractions--Sample
# of users (active/total): 589939/593578
# of items (active/total): 1264949/4340406
# of interactions: 4667861
Sparsity in the UI mat: 2e-06
Memory used by sparse UI mat: 4.45 MB
Generating train-test split..Warning: Test matrix is set as empty/all-zeros

----UserItemInteractions--Sample
# of users (active/total): 468482/593578
# of items (active/total): 1051077/4340406
# of interactions: 3409467
Sparsity in the UI mat: 1e-06
Memory used by sparse UI mat: 3.25 MB
Generating train-test split..Warning: Test matrix is set as empty/all-zeros

----UserItemInteractions--Sample
# of users (active/total): 585798/593578
# of items (active/total): 543435/4340406
# of inter

## BPR 

In [ ]:
bpr1 = BPR(
    num_features=200,
    reg_lambda=0.0,
    num_iters=200,
    learning_rate = 0.1,
    batch_size=15000,
    initial_std=0.0001,
)
bpr1.initiate(num_users=num_users, num_items=num_items)

In [ ]:
pos_data = data_ordered
neg_data = data_clicked_not_ordered
metric_log_train = []
neg_sampler = partial(
    explicit_negative_sampler,
    pos_uimat=pos_data.mat,
    neg_uimat=neg_data.mat
)

for _ in range(5):
    results = bpr_fit(
        bpr_obj=bpr1, 
        neg_sampler=neg_sampler, 
        ncores=104
    )
    mfunc = partial(
        bpr1.get_metric_v1,
        perc_active_users=0.1,
        perc_active_items=0.1,
        num_recs=60,
        max_users_per_batch=1000,
        percentiles=[0.25,0.5, 0.75],
        seed=1234
    )
    iscore = mfunc(uimat=pos_data.mat)
    metric_log_train.append(iscore)
    print(iscore)
metric_log_train = np.asarray(metric_log_train)

In [ ]:
OUT_DIR ='/projects/zazzle/rsandhu/pybpr/examples/output'
bpr1.save_model(dir_name=OUT_DIR)
bpr1.release_shm()

In [ ]:
bpr1.load_model(OUT_DIR)

In [ ]:
bpr2 = BPR()
bpr2.load_model(OUT_DIR)

In [ ]:
np.all(bpr2.umat == bpr1.umat)
np.all(bpr2.imat == bpr1.imat)

In [ ]:
bpr2

In [ ]:
bpr1